# 模型文件地址
    
    https://github.com/PaddlePaddle/models/tree/develop/PaddleCV/image_classification
    
    wget http://paddle-imagenet-models-name.bj.bcebos.com/ResNet50_pretrained.tar
    tar -xvf ResNet50_pretrained.tar
    
# 初始化环境

    pip install opencv-python
    pip install paddlepaddle==1.5
    
    

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import cv2
import time
import sys
import math
import numpy as np
import argparse
import functools

import paddle
import paddle.fluid as fluid

#加载自定义文件
from sdk.resnet import ResNet50
from sdk.attack_pp import *


In [2]:
"""
use_gpu=False

x = fluid.layers.data(name="x", shape=[1] ,dtype='int32')
x.stop_gradient=False
y = x*3

g = fluid.backward.gradients(targets=y, inputs=[x])[0]

place = fluid.CUDAPlace(0) if use_gpu else fluid.CPUPlace()
exe = fluid.Executor(place)
exe.run(fluid.default_startup_program())

result = exe.run(fluid.default_main_program(),
                 fetch_list=[y.name],
                 feed={ 'x':np.array([1]).astype('int32') })
#result = result[0][0]

print(result)
"""

'\nuse_gpu=False\n\nx = fluid.layers.data(name="x", shape=[1] ,dtype=\'int32\')\nx.stop_gradient=False\ny = x*3\n\ng = fluid.backward.gradients(targets=y, inputs=[x])[0]\n\nplace = fluid.CUDAPlace(0) if use_gpu else fluid.CPUPlace()\nexe = fluid.Executor(place)\nexe.run(fluid.default_startup_program())\n\nresult = exe.run(fluid.default_main_program(),\n                 fetch_list=[y.name],\n                 feed={ \'x\':np.array([1]).astype(\'int32\') })\n#result = result[0][0]\n\nprint(result)\n'

# 创建对抗样本相关的损失函数和标签并加载模型

In [3]:
#http://www.paddlepaddle.org.cn/documentation/docs/zh/1.5/user_guides/howto/prepare_data/index_cn.html

#fluid.default_startup_program().random_seed = 1
#fluid.default_main_program().random_seed = 1

class_dim=1000
image_shape=[3,224,224]
model_name="resnet50"
use_gpu=False
pretrained_model="ResNet50_pretrained"
TOPK=3

adv_program=fluid.Program()

with fluid.program_guard(adv_program):
    input_layer = fluid.layers.data(name='image', shape=image_shape, dtype='float32')
    #设置为可以计算梯度
    input_layer.stop_gradient=False

    # model definition
    model = ResNet50()
    out = model.net(input=input_layer, class_dim=class_dim)
    out = fluid.layers.softmax(out)

    place = fluid.CUDAPlace(0) if use_gpu else fluid.CPUPlace()
    exe = fluid.Executor(place)
    exe.run(fluid.default_startup_program())

    #记载模型参数
    fluid.io.load_persistables(exe, pretrained_model)
    
    
#创建测试用评估模式
eval_program = adv_program.clone(for_test=True)

with fluid.program_guard(adv_program):
    label = fluid.layers.data(name="label", shape=[1] ,dtype='int64')
    loss = fluid.layers.cross_entropy(input=out, label=label)
    #http://www.paddlepaddle.org.cn/documentation/docs/zh/1.5/api_cn/backward_cn.html
    gradients = fluid.backward.gradients(targets=loss, inputs=[input_layer])[0]


# 预测原始图片

In [4]:
img_path = "mug227.png"

img=process_img(img_path)


fetch_list = [out.name]

print(img.shape)

result = exe.run(eval_program,
                 fetch_list=fetch_list,
                 feed={ 'image':img })
result = result[0][0]

pred_label = np.argsort(result)[::-1][:TOPK]
print("Test-score: {0}, class {1}".format(result[pred_label], pred_label))
sys.stdout.flush()

(1, 3, 224, 224)
Test-score: [0.63326067 0.23805845 0.0749701 ], class [504 967 968]


# 调用FGSM算法 无定向攻击

In [5]:
img_path = "mug227.png"
img=process_img(img_path)

#init_prog(fluid.default_main_program())

#生成对抗样本
adv=FGSM(adv_program=adv_program,eval_program=eval_program,gradients=gradients,o=img,input_layer=input_layer,output_layer=out,step_size=16.0/256.0,isTarget=False,target_label=0)

    
result = exe.run(eval_program,
                 fetch_list=fetch_list,
                 feed={ 'image':adv })
result = result[0][0]

pred_label = np.argsort(result)[::-1][:TOPK]

print("Test-score: {0}, class {1}".format(result[pred_label], pred_label))
sys.stdout.flush()

Non-Targeted attack target_label=o_label=504
Test-score: [0.98026764 0.00760567 0.00633906], class [967 504 968]


# 调用FGSM算法 定向攻击

In [6]:
img_path = "mug227.png"
img=process_img(img_path)

#init_prog(fluid.default_main_program())

#生成对抗样本
adv=FGSM(adv_program=adv_program,eval_program=eval_program,gradients=gradients,o=img,
         input_layer=input_layer,output_layer=out,step_size=16.0/256.0,isTarget=True,target_label=968)

    
result = exe.run(eval_program,
                 fetch_list=fetch_list,
                 feed={ 'image':adv })
result = result[0][0]

pred_label = np.argsort(result)[::-1][:TOPK]

print("Test-score: {0}, class {1}".format(result[pred_label], pred_label))
sys.stdout.flush()

Targeted attack target_label=968 o_label=504
Test-score: [0.48446572 0.31735182 0.07795694], class [968 504 967]
